In [1]:
# Create an instance of Candidate that implements the xValue property and
# initialise the game with your implementation

from holdet.game import Game, BaseCandidate, Holdet, Sofascore


class CandidateEMA(BaseCandidate):
    def __init__(self, holdet: Holdet, sofascore: Sofascore) -> None:
        super().__init__(holdet, sofascore)

    def xGrowth(self, alpha: float) -> float:
        """
        Predict the growth for the next round using exponential moving average
        (EMA). Set alpha to adjust the smoothing factor, between 0 and 1. Higher
        values give more weight to recent stats.
        """
        ema = 0.0
        for i, round in enumerate(sorted(self.rounds)):
            if i == 0:
                ema = round.xGrowth
            else:
                ema = alpha * round.xGrowth + (1 - alpha) * ema
        return ema

    @property
    def xValue(self) -> float:
        if self.captain:
            return self.value + self.xGrowth(0.5) * 2
        return self.value + self.xGrowth(0.5)


game = Game(CandidateEMA)


In [2]:
# Set your budget and calculate the starting 11 that will maximize the expected
# value (xValue) within the rules of the game.

from rich.jupyter import print
from holdet.solver import lp
from holdet.formation import Formation

budget = 50 * 1000000  # 50 million

solution = lp.find_optimal_team(game.candidates, budget)
print(Formation(solution))


                                                                             
                                 XI (3-4-3)                                  
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Position    ┃ Players                                           ┃ xGrowth ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ Keeper      │ ⚽️ Kasper Schmeichel (Leicester City), value=0.0M │     32K │
│             │                                                   │         │
│ Defenses    │ ⚽️ Christian Marques (Wolverhampton), value=0.0M  │     98K │
│             │ ⚽️ Brandon Williams (Manchester Utd), value=1.9M  │    181K │
│             │ ⚽️ Ethan Pinnock (Brentford), value=4.1M          │    168K │
│             │                                                   │         │
│ Midfielders │ ⚽️ Jacob Ramsey (Aston Villa), value=5.0M         │    258K │
│             │ ⚽️ Bryan Mbeumo (Brentford), value=6.1M           │    244K │
│             │ ⚽️ Cole Palmer (Manchester City), value=3.2M      │    237K │
│             │ 👑 Bruno Fernandes (Manchester Utd), value=8.8M   │    646K │
│             │                                                   │         │
│ Forwards    │ ⚽️ Aleksandar Mitrovic (Fulham), value=6.8M       │    257K │
│             │ ⚽️ Ollie Watkins (Aston Villa), value=7.6M        │    264K │
│             │ ⚽️ Jadon Sancho (Manchester Utd), value=6.1M      │    227K │
│ Total       │ value=49.5M                                       │   2611K │
└─────────────┴───────────────────────────────────────────────────┴─────────┘